In [1]:
import xgboost as xgb
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import csv

In [2]:
def show_accuracy(a, b, tip):
    acc = a.ravel() == b.ravel()
    acc_rate = 100 * float(acc.sum()) / a.size
    # print '%s正确率：%.3f%%' % (tip, acc_rate)
    return acc_rate

In [3]:
data=pd.read_csv(r'F:\study\ml\DoctorZou\11Boost\12.XGBoost\12.Titanic.train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#sex
data['Sex']=data['Sex'].map({'female':0,'male':1}).astype(int)

In [5]:
if len(data.Fare[data.Fare.isnull()])>0:
    fare =np.zeros(3)
    for f in range(0,3):
        fare[f]=data[data.Pclass==f+1]['Fare'].dropna().median()
    for f in range(0,3):
        data.loc[(data.Fare.isnull()) & (data.Pclass == f+1),'Fare'] =fare[f]

In [13]:
# use RF predict age
data_for_age=data[['Age','Survived','Fare','Parch','SibSp','Pclass']]
age_exist=data_for_age.loc[(data.Age.notnull())]
age_null=data_for_age.loc[(data.Age.isnull())]

x=age_exist.values[:,1:]
y=age_exist.values[:,0]


In [14]:
data.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.647587,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,1.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200


In [15]:
rfr=RandomForestRegressor(n_estimators=1000)
rfr.fit(x,y)
age_hat=rfr.predict(age_null.values[:,1:])

In [18]:
data.loc[(data.Age.isnull()),'Age']=age_hat

In [19]:
#city
data.loc[(data.Embarked.isnull()),'Embarked']='S'
embarked_data=pd.get_dummies(data.Embarked)

In [21]:
embarked_data.head()

,C,Q,S,U
0,0,0,1,0
1,1,0,0,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0


In [22]:
embarked_data=embarked_data.rename(columns=lambda x:'Embarked_'+str(x))
embarked_data.head()

,Embarked_C,Embarked_Q,Embarked_S,Embarked_U
0,0,0,1,0
1,1,0,0,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0


In [23]:
data=pd.concat([data,embarked_data],axis=1)
data.to_csv(r'F:\study\ml\DoctorZou\11Boost\12.XGBoost\New_Data.csv')

In [24]:
    x = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S']]

In [27]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Embarked_C,Embarked_Q,Embarked_S,Embarked_U
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,7.2500,NaN,S,0,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,PC 17599,71.2833,C85,C,1,0,0,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,53.1000,C123,S,0,0,1,0
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,8.0500,NaN,S,0,0,1,0
5,6,0,3,"Moran, Mr. James",1,23.965656,0,0,330877,8.4583,NaN,Q,0,1,0,0
6,7,0,1,"McCarthy, Mr. Timothy J",1,54.000000,0,0,17463,51.8625,E46,S,0,0,1,0
7,8,0,3,"Palsson, Master. Gosta Leonard",1,2.000000,3,1,349909,21.0750,NaN,S,0,0,1,0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",0,27.000000,0,2,347742,11.1333,NaN,S,0,0,1,0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",0,14.000000,1,0,237736,30.0708,NaN,C,1,0,0,0


In [25]:
y=None
if 'Survived' in data:
    y=data['Survived']

In [26]:
for i in data:
    print(i)

PassengerId
Survived
Pclass
Name
Sex
Age
SibSp
Parch
Ticket
Fare
Cabin
Embarked
Embarked_C
Embarked_Q
Embarked_S
Embarked_U


In [ ]:
x=np.array(x)
y=np.array(y)
x=np.tile(x,(5,1))
y=np.tile(y,(5,1))

In [29]:
def write_result(c,c_type):
    if tyep==3:
        x=xgb.DMatrix(x)
    y=c.predict(x)
    y[y>0.5]=1
    y[~(y>0.5)]=0
    
    predictions_file = open("Prediction_%d.csv" % c_type, "wb")
    open_file_object = csv.writer(predictions_file)
    open_file_object.writerow(["PassengerId", "Survived"])
    open_file_object.writerows(zip(passenger_id, y))
    predictions_file.close()
    

In [30]:
x.shape

(891, 9)

In [32]:
y.shape

(891,)

In [34]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.5,
                                                random_state=1)

In [37]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
394,3,0,24.00,0,2,16.7000,0,0,1
851,3,1,74.00,0,0,7.7750,0,0,1
373,1,1,22.00,0,0,135.6333,1,0,0
523,1,0,44.00,0,1,57.9792,1,0,0
78,2,1,0.83,0,2,29.0000,0,0,1


In [38]:
##lr
lr=LogisticRegression(penalty='l2')
lr.fit(x_train,y_train)
y_hat=lr.predict(x_test)
lr_rate=show_accuracy(y_hat,y_test,'Logistic Regression ')
lr_rate

77.80269058295964

In [39]:
## RFC
rfc=RandomForestClassifier(n_estimators=1000)
rfc.fit(x_train,y_train)
y_hat=rfc.predict(x_test)
rfc_rate=show_accuracy(y_hat,y_test,'RF')
rfc_rate

80.26905829596413

In [40]:
#xgb
data_train=xgb.DMatrix(x_train,label=y_train)
data_test=xgb.DMatrix(x_test,label=y_test)
watch_list=[(data_test,'eval'),(data_train,'train')]
param={'max_depth':3,'eta':0.1,'silent':1,'objective':'binary:logistic'}
bst=xgb.train(param,data_train,num_boost_round=100,evals=watch_list)
y_hat=bst.predict(data_test)
y_hat[y_hat>0.5]=1
y_hat[~(y_hat>0.5)]=0
xgb_rate=show_accuracy(y_hat,y_test,'XGB ')
xgb_rate

[17:37:40] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	eval-error:0.20404	train-error:0.13933
[1]	eval-error:0.20404	train-error:0.13933
[2]	eval-error:0.20404	train-error:0.13933
[3]	eval-error:0.20404	train-error:0.14157
[4]	eval-error:0.20404	train-error:0.14157
[5]	eval-error:0.20404	train-error:0.14157
[6]	eval-error:0.20404	train-error:0.14157
[7]	eval-error:0.20852	train-error:0.14157
[8]	eval-error:0.20404	train-error:0.12584
[9]	eval-error:0.20404	train-error:0.12584
[10]	eval-error:0.20628	train-error:0.12360
[11]	eval-error:0.21076	train-error:0.12360
[12]	eval-error:0.21525	train-error:0.13933
[13]	eval-error:0.21300	train-error:0.12809
[14]	eval-

80.71748878923766

In [43]:
pd.DataFrame([1,2,100]).median()

0    2.0
dtype: float64

In [44]:
pd.DataFrame([1,2,100]).mean()

0    34.333333
dtype: float64

In [45]:
pd.DataFrame([1,2,100]).std()

0    56.8712
dtype: float64

In [46]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Embarked_C,Embarked_Q,Embarked_S,Embarked_U
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C,1,0,0,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S,0,0,1,0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S,0,0,1,0
